In [31]:
# import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import load_model
import statsmodels.api as sm
from scipy import stats

In [41]:
# load CSVs
##Life Expectancy Data
life = pd.read_csv('Resources/lifeexpectancy.csv')

## SVI 2014 Data
svi14 = pd.read_csv('Resources/SvI2014_US_CNTY.csv')

## SVI 2010 Data
svi10 = pd.read_csv('Resources/SVI2010_US.csv')

## SVI 2000 Data
svi00 = pd.read_csv('Resources/SVI2000_US.csv')

In [42]:
############# GET LIFE EXPECTANCY DATA #########################

# remove column, drop index
life.columns = life.iloc[0]
life = life.reindex(life.index.drop(0))

# clean up columns
life = life.rename(columns={'Life expectancy, 2014*': 'Life2014',
                            'Life expectancy, 2010*': 'Life2010',
                            'Life expectancy, 2000*': 'Life2000'})
life = life[['Location', 'FIPS', 'Life2000', 'Life2010', 'Life2014']]
life.head()

# take first five digits from life expectancy column
life['Life2014'] = life['Life2014'].str[:5]
life['Life2010'] = life['Life2010'].str[:5]
life['Life2000'] = life['Life2000'].str[:5]

# change FIPS to integer for merge
life['FIPS'] = pd.to_numeric(life['FIPS'], errors='coerce')
life = life.dropna(subset=['FIPS'])
life['FIPS'] = life['FIPS'].astype(int)

# remove columns that just have state overall data (state data column in FIPS are all under the value of 100)
life = life[life.FIPS > 100]
life = life.reset_index(drop=True)

# create data frames for later merging
life2000 = life[['FIPS', 'Life2000']]
life2010 = life[['FIPS', 'Life2010']]
life2014 = life[['FIPS', 'Life2014']]

life2000.head()

,FIPS,Life2000
0,1001,74.55
1,1003,76.80
2,1005,74.13
3,1007,73.56
4,1009,75.62


In [43]:
################### 2014 SVI DATA ###################

# merge life expectancy data with 2014 SVI data
svi14 = svi14.merge(life2014, how="outer", left_on="FIPS", right_on='FIPS')

# rename column and add year column
svi14 = svi14.rename(columns={'Life2014': 'LifeExpectancy'})
svi14['Year'] = 2014

### limit dataframe to SVI categories and themes
svi14 = svi14[['COUNTY', 'FIPS', 'EP_POV', 'EP_UNEMP', 'EP_PCI', 'EP_NOHSDP', 'EP_AGE65', 'EP_AGE17', 'EP_DISABL',
'EP_SNGPNT', 'EP_MINRTY', 'EP_LIMENG', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH',
'EP_GROUPQ', 'RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES', 'LifeExpectancy', "Year"]]

# dtype check reveals that LifeExpectancy is an object, so change it to numeric
svi14["LifeExpectancy"] = pd.to_numeric(svi14.LifeExpectancy, errors='coerce')

# NaN check reveals that two FIPS codes only have life expectancy data (the two FIPS codes were discontinued)
svi14 = svi14[svi14.RPL_THEMES >= 0]
svi14 = svi14[svi14.LifeExpectancy >= 0]

# rename the columns so the make sense
svi14 = svi14.rename(columns={'EP_POV': "Poverty", 'EP_PCI': "Per_Capita_Income",
                 'EP_NOHSDP': "NoHighSchoolDiploma", 'EP_AGE65': 'Above64', 'EP_AGE17': 'Below18',
                 'EP_DISABL': 'Disability', 'EP_SNGPNT': 'SingleParent', 'EP_MINRTY': 'NonWhite',
                 'EP_LIMENG': 'LimitedEnglish', 'EP_MUNIT': 'Housing9Units', 'EP_MOBILE': "MobileHomes",
                 'EP_CROWD': 'MorePeopleThanRooms', 'EP_NOVEH': 'NoVehicle', 'EP_GROUPQ': 'Institutionalized',
                 'RPL_THEME1': "SocioeconomicSVI", 'RPL_THEME2': 'HouseCompSVI', 'RPL_THEME3': 'MinorityLangSVI',
                 'RPL_THEME4': 'HousingTransportSVI', 'RPL_THEMES': 'OverallSVI'})

svi14.head()

,COUNTY,FIPS,Poverty,EP_UNEMP,Per_Capita_Income,NoHighSchoolDiploma,Above64,Below18,Disability,SingleParent,...,MorePeopleThanRooms,NoVehicle,Institutionalized,SocioeconomicSVI,HouseCompSVI,MinorityLangSVI,HousingTransportSVI,OverallSVI,LifeExpectancy,Year
0,Autauga,1001,12.8,8.5,24644.0,13.8,13.3,25.8,16.0,8.1,...,2.6,5.3,0.8,0.4145,0.5387,0.4986,0.5100,0.4696,75.67,2014
1,Baldwin,1003,13.8,8.6,26851.0,11.0,17.7,22.6,14.1,6.9,...,1.4,3.1,1.4,0.3372,0.2811,0.6288,0.3238,0.3432,78.08,2014
2,Barbour,1005,24.1,14.2,17350.0,25.4,15.4,21.6,20.0,12.0,...,1.9,8.8,10.6,0.9628,0.8195,0.8898,0.9089,0.9742,75.42,2014
3,Bibb,1007,17.0,10.9,18110.0,22.1,14.2,21.6,15.8,9.0,...,0.2,4.2,7.0,0.8201,0.2948,0.5237,0.4152,0.6278,73.97,2014
4,Blount,1009,17.3,9.3,20501.0,21.9,15.9,24.1,17.0,7.9,...,1.6,3.9,1.0,0.7348,0.6199,0.5864,0.1563,0.5581,76.16,2014


In [44]:
len(svi14)

3140

In [45]:
svi14 = svi14[["FIPS", "Per_Capita_Income", "Poverty", "Disability", "NoHighSchoolDiploma", "LifeExpectancy"]]

In [46]:
svi14.head()

,FIPS,Per_Capita_Income,Poverty,Disability,NoHighSchoolDiploma,LifeExpectancy
0,1001,24644.0,12.8,16.0,13.8,75.67
1,1003,26851.0,13.8,14.1,11.0,78.08
2,1005,17350.0,24.1,20.0,25.4,75.42
3,1007,18110.0,17.0,15.8,22.1,73.97
4,1009,20501.0,17.3,17.0,21.9,76.16


In [49]:
# save the svi2014 data 
svi14.to_csv('Output/svi14final.csv')

In [48]:
len(svi14)

3140

In [51]:
svi00 = svi00[['STCOFIPS', 'Totpop2000', 'Totalhu', "G1V3R", 'G2V4N', "G1V1N", 'G4V2N']]

In [52]:
svi00.head()

,STCOFIPS,Totpop2000,Totalhu,G1V3R,G2V4N,G1V1N,G4V2N
0,1001,1879,742,17771,73,227,154
1,1001,1934,758,14217,74,433,15
2,1001,3339,1263,18346,136,250,18
3,1001,4556,1871,19741,98,207,92
4,1001,6040,2277,24510,143,222,16


In [55]:
# rename columns
svi00 = svi00.rename(columns={'STCOFIPS': 'FIPS',
                              'Totpop2000': 'Pop',
                              'Totalhu': 'HH',
                              "G1V3R": 'Per_Capita_Income1',
                              'G1V1N': 'Poverty1',
                              'G2V4N': 'SingleParent1',
                              'G4V2N': 'MobileHomes1'})

# calculate total income based on census tract
svi00['TotalIncome'] = svi00['Pop'] * svi00['Per_Capita_Income1']

# groupby FIPS and sum the rest of columns
svi00 = svi00.groupby(['FIPS']).sum()

# calculate the PCI, poverty, singleparent, mobilehomes for 
svi00['Per_Capita_Income'] = (svi00['TotalIncome'] / svi00['Pop']) 
svi00['Poverty'] = (svi00['Poverty1'] / svi00['Pop']) * 100
svi00['SingleParent'] = (svi00['SingleParent1'] / svi00['HH']) * 100
svi00['MobileHomes'] = (svi00['MobileHomes1'] / svi00['HH']) * 100

svi00.head()

,Pop,HH,Per_Capita_Income1,SingleParent1,Poverty1,MobileHomes1,TotalIncome,Per_Capita_Income,Poverty,SingleParent,MobileHomes
FIPS,,,,,,,,,,,
1001,43671,17662,194948,1520,4738,4615,8513574108,194948.0,10.849305,8.606047,26.129544
1003,140415,74285,476906,3974,14018,13127,66964755990,476906.0,9.983264,5.349667,17.671131
1005,29038,12461,123501,1356,7032,3687,3586222038,123501.0,24.216544,10.881952,29.588316
1007,20826,8345,42691,624,4091,2839,889082766,42691.0,19.643715,7.477531,34.020371
1009,51024,21158,129437,1102,5930,6026,6604393488,129437.0,11.621982,5.208432,28.480953


In [23]:
############################# SVI 2010 Data ##############################

# length check reveals that data is by census block and not county 
# documentation reveals that -999 values are actually NaN, so remove those rows
svi10 = svi10[svi10.E_P_POV >= 0]

#rename of all potential columns for analysis to match 2014 SVI data names
svi10 = svi10.rename(columns = {'E_P_POV': "EP_POV",
                                'E_P_UNEMP': "EP_UNEMP", 
                                'E_P_NOHSDP': 'EP_NOHSDP', 
                                'P_AGE65': 'EP_AGE65',
                                'P_AGE17': 'EP_AGE17', 
                                'P_SNGPRNT': 'EP_SNGPNT', 
                                'P_MINORITY': 'EP_MINRTY',
                                'E_P_LIMENG': 'EP_LIMENG',
                                'E_P_MUNIT': 'EP_MUNIT', 
                                'E_P_MOBILE': 'EP_MOBILE', 
                                'E_P_CROWD': 'EP_CROWD',
                                'E_P_NOVEH': 'EP_NOVEH', 
                                'P_GROUPQ': 'EP_GROUPQ', 
                                'R_PL_THEME1': 'RPL_THEME1', 
                                'R_PL_THEME2': 'RPL_THEME2',
                                'R_PL_THEME3': 'RPL_THEME3',
                                'R_PL_THEME4': 'RPL_THEME4', 
                                'R_PL_THEMES': 'RPL_THEMES', 
                                'E_P_PCI': 'EP_PCI',
                                'E_P_NOHSDIP': "EP_NOHSDP", 
                                'FIPS': "CT", 
                                'STCOFIPS': 'FIPS'})

# add year column
svi10['Year'] = 2010

# gather total income by census tract before groupby
svi10['TotalIncome'] = svi10['TOTPOP'] * svi10['E_PCI']

# 2010 SVI data is available only by census tract so need to group by FIPS and sum
svi10 = svi10.groupby(['FIPS']).sum()

# after testing data thru linear regression only pull important columns
# and columns where the ability to create by county data is possible
svi10['FIPS'] = svi10.index
svi10 = svi10[['FIPS', 'TOTPOP', 'HH', 'E_HU', 'E_POV', 'E_PCI', 'SNGPRNT', 'E_MOBILE', 'TotalIncome']]

# create columns for county-based data
svi10['Per_Capita_Income'] = (svi10['TotalIncome'] / svi10['TOTPOP'])
svi10['Poverty'] = (svi10['E_POV'] / svi10['TOTPOP']) * 100
svi10['SingleParent'] = (svi10['SNGPRNT'] / svi10['HH']) * 100
svi10['MobileHomes'] = (svi10['E_MOBILE'] / svi10['HH']) * 100

# clean-up columns to display just what is wanted
svi10 = svi10[['FIPS', "Per_Capita_Income", "Poverty", "SingleParent", "MobileHomes"]]

# make FIPS a column and reset index
svi10 = svi10.reset_index(drop=True)

# save svi10 for future use
svi10.to_csv('Resources/svi10limit.csv')

svi10.head()

,FIPS,Per_Capita_Income,Poverty,SingleParent,MobileHomes
0,1001,24644.103883,10.304008,11.819396,22.827753
1,1003,26257.835750,11.640194,9.788193,19.764963
2,1005,15801.342936,22.591689,16.323829,36.680244
3,1007,19882.204495,12.349989,12.146360,33.521941
4,1009,21092.635672,13.101427,8.647697,24.460098
